In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from category_encoders import BinaryEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix, f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm
from collections import defaultdict
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import string
import matplotlib.pyplot as plt
import pickle
import joblib
from joblib import load

# pd.set_option('display.max_rows', None) 
# pd.set_option('display.max_columns', None) 
# pd.set_option('display.width', None) 

## EDA

In [3]:
# Modified Dataset
df = pd.read_csv('Disease_Dataset_Modified.csv')

In [4]:
df

,label_dis,abdominal cramp,abdominal distention,abnormal behavior,abnormal bleeding,abnormal sensation,abnormally frequent,abscess,aching,acne,...,wet,wheezing,white patch vaginal discharge,widespread pain,withdrawal occurring stopping,worrying,yellow skin,yellowish coloration skin white eye,yellowish skin,yellowish skin crust
0,Abscess,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abscess,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abscess,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abscess,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abscess,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8830,papilloedema,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8831,papilloedema,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8832,papilloedema,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8833,papilloedema,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.rename(columns={'label_dis': 'Disease'}, inplace=True) # Rename
df=df.applymap(lambda x: x.strip().title() if isinstance(x, str) else x)
df.columns = [col.strip().title() for col in df.columns] # strip() + title() for all the Symptoms or Columns

In [6]:
# drop redundant features
df = df.drop(columns=['Blue', 'Wet', 'Others', 'Index', 'Jaw', 'May Symptom', 'Middle Finger', 'Minimal', 'Painful', 'Red', 'Sit', 'Variable','Arm','Back','Neck'])

In [7]:
df.shape

(8835, 475)

In [9]:
df['Disease'] = df['Disease'].replace('Celiacs Disease', 'Coeliac Disease')
df['Disease'] = df['Disease'].replace('Gerd', 'GERD')
df['Disease'] = df['Disease'].replace('Middle East Respiratory Syndrome Coronavirus (Mers‐Cov)', 'MERS-COV')

In [10]:
df = df[df.eq(1).sum(axis=1) > 1]

In [11]:
df.shape

(7740, 475)

In [12]:
df.head()

,Disease,Abdominal Cramp,Abdominal Distention,Abnormal Behavior,Abnormal Bleeding,Abnormal Sensation,Abnormally Frequent,Abscess,Aching,Acne,...,Weight Gain,Wheezing,White Patch Vaginal Discharge,Widespread Pain,Withdrawal Occurring Stopping,Worrying,Yellow Skin,Yellowish Coloration Skin White Eye,Yellowish Skin,Yellowish Skin Crust
3,Abscess,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abscess,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Abscess,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Abscess,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,Acute Encephalitis Syndrome,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df['Disease'].nunique()

213

In [14]:
df.isnull().values.any()

False

In [15]:
#Frequency distribution of classes"
freq_distribution = pd.crosstab(index=df["Disease"], columns="count")  

freq_distribution

col_0,count
Disease,
Abscess,4
Acute Encephalitis Syndrome,26
Adult Inclusion Conjunctivitis,1
Alcohol Abuse And Alcoholism,26
Alzheimer,11
...,...
Warkany Syndrome,11
Warts,4
Yaws,4


## Observation
- 213 unique disease and 474 unique symptoms
- 7740 rows
- no Null values

## Train Test Split

In [16]:
X = df.drop(['Disease'], axis=1)
Y = df['Disease']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.20, random_state=42)

## Train RF

In [18]:
rf = RandomForestClassifier(n_estimators = 110)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=110)

In [ ]:
symptoms_list = list(X.columns)

In [89]:
len(symptoms_list)

474

## Evaluation RF

In [45]:
# Cross Validation Accuracy
def cvaccuracy(model):
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    cvaccuracy = cross_val_score(model, X, Y, scoring='accuracy', cv = skf)
    print("Cross Validation Accuracy ",cvaccuracy.mean())

In [24]:
cvaccuracy(rf)

Cross Validation Accuracy  0.9291989664082687


## SVM

### HyperParameter Tuning

In [37]:
svm = svm.SVC()

param_grid = {'C': [0.1, 1, 5, 10, 100],
              'gamma': ['scale', 'auto'],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
grid_svm = GridSearchCV(svm,param_grid,scoring='accuracy', cv=5)

In [38]:
grid_svm.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 5, 10, 100], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='accuracy')

In [39]:
print(grid_svm.best_params_)

{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}


### One Vs One

In [36]:
from sklearn.multiclass import OneVsOneClassifier
svm_classifier_ovo_rbf = svm.SVC(kernel='rbf', probability=True)

ovo_classifier_rbf = OneVsOneClassifier(svm_classifier_ovo_rbf)

ovo_classifier_rbf.fit(X_train, y_train)

predictions_rbf = ovo_classifier_rbf.predict(X_test)

In [26]:
print("One vs One (rbf) Test Accuracy ", accuracy_score(predictions_rbf, y_test))

One vs One (rbf) Test Accuracy  0.915374677002584


In [51]:
print("One vs One (rbf) CV Accuracy ", cvaccuracy(ovo_classifier_rbf))

Cross Validation Accuracy  0.9370801033591732
One vs One (rbf) CV Accuracy  None


In [52]:
pred_ovo_train_rbf = ovo_classifier_rbf.predict(X_train)
train_accuracy_ovo_rbf = accuracy_score(pred_ovo_train_rbf, y_train)
print("One vs One (rbf) Train Accuracy ", train_accuracy_ovo_rbf)

One vs One (rbf) Train Accuracy  0.9526808785529716


### One vs Many

In [42]:
from sklearn.multiclass import OneVsRestClassifier

svm_classifier_ova_rbf = svm.SVC(kernel='rbf',probability=True)
ova_classifier_rbf = OneVsRestClassifier(svm_classifier_ova_rbf)

ova_classifier_rbf.fit(X_train, y_train)

pred_ova_rbf = ova_classifier_rbf.predict(X_test)

In [43]:
print("One vs Many (rbf) Test Accuracy ", accuracy_score(pred_ova_rbf, y_test))

One vs Many (rbf) Test Accuracy  0.9257105943152455


In [44]:
cvaccuracy(ova_classifier_rbf)

Cross Validation Accuracy  0.9403100775193798


In [45]:
pred_ova_train_rbf = ova_classifier_rbf.predict(X_train)
train_accuracy_rbf = accuracy_score(pred_ova_train_rbf, y_train)
print("One vs Many (rbf) Train Accuracy ", train_accuracy_rbf)

One vs Many (rbf) Train Accuracy  0.9588178294573644


## Export Models

In [74]:
joblib.dump(ova_classifier_rbf, 'svm_model_file.gz',compress=3) #SVM One vs All

['svm_model_file.gz']

In [37]:
joblib.dump(ovo_classifier_rbf, 'svm_model_ovo_file.gz',compress=3)

['svm_model_ovo_file.gz']

In [231]:
joblib.dump(rf,  'rf_model_file.gz',compress=3) # (n_estimators=110)

['rf_model_file.gz']

In [230]:
joblib.dump(symptoms_list, 'symptoms_file.gz',compress=3)

['symptoms_file.gz']

In [39]:
df_specialist_reset_index = df_specialist.reset_index(drop=True)
joblib.dump(df_specialist_reset_index,  'specialist_file.gz',compress=3)

['specialist_file.gz']

## Evaluation

In [28]:
def evaluate_model(model, X_test, y_test):
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')
  f1 = f1_score(y_test, y_pred, average='weighted')
  return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [29]:
rf_results = evaluate_model(loaded_rf, X_test, y_test)
print("Random Forest Model Results:")
print(rf_results)

svm_ovo_results = evaluate_model(loaded_svm_ovo, X_test, y_test)
print("SVM OVO Model Results:")
print(svm_ovo_results)

svm_ova_results = evaluate_model(loaded_svm_ova, X_test, y_test)
print("SVM OVA Model Results:")
print(svm_ova_results)

Random Forest Model Results:
{'accuracy': 0.9192506459948321, 'precision': 0.9246275148775995, 'recall': 0.9192506459948321, 'f1': 0.9171742812815586}
SVM OVO Model Results:
{'accuracy': 0.915374677002584, 'precision': 0.8934120965932035, 'recall': 0.915374677002584, 'f1': 0.8967872055174444}
SVM OVA Model Results:
{'accuracy': 0.9257105943152455, 'precision': 0.92781891154259, 'recall': 0.9257105943152455, 'f1': 0.9203061206705283}


In [9]:
y_train_pred = loaded_rf.predict(loaded_X_train)
train_accuracy = accuracy_score(y_train_pred, loaded_Y_train)
print("RF Train Accuracy ", train_accuracy)

RF Train Accuracy  0.962532299741602


## Map The Specialist

In [35]:
df_specialist = pd.read_csv('Disease_Specialist.csv', encoding='latin1')

In [37]:
df_specialist=df_specialist.applymap(lambda x: x.strip().title() if isinstance(x, str) else x)

In [38]:
df_specialist['Category'].unique()

array(['Mild To Moderate', 'Critical', 'Moderate', 'Mild To Critical',
       'Moderate To Critical', 'Mild'], dtype=object)

In [25]:
category_counts = df_specialist['Category'].value_counts()
category_counts

Category
Mild To Moderate        64
Critical                54
Mild To Critical        41
Moderate                26
Moderate To Critical    15
Mild                    13
Name: count, dtype: int64

In [26]:
df_specialist['Disease'] = df_specialist['Disease'].replace('Guillain-Barrã© Syndrome', 'Guillain-Barré Syndrome')
df_specialist['Disease'] = df_specialist['Disease'].replace('Gerd', 'GERD')
df_specialist['Disease'] = df_specialist['Disease'].replace('Mers?Cov', 'MERS-COV')
df_specialist['Disease'] = df_specialist['Disease'].replace('Kaposi\x92S Sarcoma', 'Kaposi’S Sarcoma')

In [27]:
df_specialist.head()

,Disease,Specialist,Category
0,Abscess,Dermatologist,Mild To Moderate
1,Acute Encephalitis Syndrome,Neurologist,Critical
2,Adult Inclusion Conjunctivitis,"Ophthalmologist, Id Specialist",Mild To Moderate
3,Alcohol Abuse And Alcoholism,Psychiatrist,Moderate
4,Alzheimer,Neurologist,Critical


## Predicting Top 3 Results

### Load required files

In [19]:
loaded_rf = load('rf_model_file.gz')
loaded_svm_ovo = load('svm_model_ovo_file.gz')
loaded_svm_ova = load('svm_model_file.gz')

loaded_symptoms_list = load('symptoms_file.gz')
loaded_df_specialist = load('specialist_file.gz')

In [20]:
def find_specialist(disease):
    specialist = loaded_df_specialist .loc[loaded_df_specialist ['Disease'] == disease, 'Specialist'].values
    if len(specialist) > 0:
        return f"{str(specialist[0])}"
    else:
        return "No Specialist"

In [21]:
def find_category(disease):
    category = loaded_df_specialist .loc[loaded_df_specialist ['Disease'] == disease, 'Category'].values
    if len(category) > 0:
        return f"{str(category[0])}"
    else:
        return "Unknown"

### User Input

In [22]:
user_symptoms = ['Abscess', 'Chest Pain', 'Diarrhea', 'Fever', 'Nausea']

# Convert user input into the format expected by the model (1 if the symptom is present in user input, 0 otherwise)
user_input_vector = [0 for _ in range(len(loaded_symptoms_list))]
for val in user_symptoms:
    if val in loaded_symptoms_list:
        user_input_vector[loaded_symptoms_list.index(val)] = 1

# Predict top 3 diseases based on the input symptoms and their probabilities
predicted_probabilities = loaded_svm_ova.predict_proba([user_input_vector])[0]
top_3_indices = sorted(range(len(predicted_probabilities)), key=lambda i: predicted_probabilities[i], reverse=True)[:3]

# Extract top 3 diseases and their probabilities
top_3_diseases = [loaded_svm_ova.classes_[i].strip().title() for i in top_3_indices]
top_3_probabilities = [predicted_probabilities[i] for i in top_3_indices]


top3_dict = defaultdict(list)
for disease, probability in zip(top_3_diseases, top_3_probabilities):
    specialist = find_specialist(disease)
    category = find_category(disease)
    top3_dict[disease].extend([round(probability * 100, 2), specialist, category])

top3_dict

defaultdict(list,
            {'Anthrax': [92.51, 'Dermatologist', 'Moderate To Critical'],
             'Hepatitis A': [0.32,
              'Hepatologist, Gastroenterologist',
              'Mild To Critical'],
             'Cchf': [0.27, 'Id Specialist', 'Critical']})